In [28]:
import kagglehub
import os, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix


In [12]:
#Download & Load Dataset
path = kagglehub.dataset_download("asdasdasasdas/garbage-classification")
print("Path to dataset files:", path)

dataset_path = "C:/Users/Begad/.cache/kagglehub/datasets/asdasdasasdas/garbage-classification/versions/2"
img_path = os.path.join(dataset_path, "Garbage classification", "Garbage classification")


Path to dataset files: C:\Users\Begad\.cache\kagglehub\datasets\asdasdasasdas\garbage-classification\versions\2


In [13]:
# Dataset Overview
classes = os.listdir(img_path)
num_classes = len(classes)
print(f"We have {num_classes} classes")

total_images = 0
for c in classes:
    images = os.listdir(os.path.join(img_path, c))
    total_images += len(images)
    print(f"{c}: {len(images)} images")

print(f"Total Images are: {total_images}")


We have 6 classes
cardboard: 403 images
glass: 501 images
metal: 410 images
paper: 594 images
plastic: 482 images
trash: 137 images
Total Images are: 2527


In [14]:
# Data Generators with Strong Augmentation
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    img_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    img_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [15]:
#Class Weights for Imbalance
y_train = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))


In [16]:
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomFlip("horizontal_and_vertical"),
])


In [17]:
#ResNet50 Base Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers:
    if 'conv4' in layer.name or 'conv5' in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False

In [18]:
#Custom Classification Head
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 256)               524544    
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                           

In [19]:
#Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [20]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Learning Rate Scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',   # watch validation loss
    factor=0.5,           # reduce LR by half
    patience=3,           # wait 3 epochs before reducing
    min_lr=1e-7,          # never go below this
    verbose=1
)




In [21]:
# Train with scheduler
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stop, lr_scheduler],   # add here
    class_weight=class_weights_dict
)

Epoch 1/30
64/64 [==============================] - 29s 392ms/step - loss: 2.1127 - accuracy: 0.2470 - val_loss: 1.4508 - val_accuracy: 0.4473 - lr: 1.0000e-05
Epoch 2/30
64/64 [==============================] - 25s 386ms/step - loss: 1.6151 - accuracy: 0.4224 - val_loss: 1.3016 - val_accuracy: 0.5288 - lr: 1.0000e-05
Epoch 3/30
64/64 [==============================] - 25s 388ms/step - loss: 1.3432 - accuracy: 0.5375 - val_loss: 1.1987 - val_accuracy: 0.6083 - lr: 1.0000e-05
Epoch 4/30
64/64 [==============================] - 26s 402ms/step - loss: 1.1787 - accuracy: 0.6299 - val_loss: 1.0791 - val_accuracy: 0.6879 - lr: 1.0000e-05
Epoch 5/30
64/64 [==============================] - 26s 398ms/step - loss: 0.9873 - accuracy: 0.7312 - val_loss: 1.0371 - val_accuracy: 0.7237 - lr: 1.0000e-05
Epoch 6/30
64/64 [==============================] - 25s 397ms/step - loss: 0.9177 - accuracy: 0.7678 - val_loss: 0.9580 - val_accuracy: 0.7455 - lr: 1.0000e-05
Epoch 7/30
64/64 [======================

In [22]:
#Fine-Tuning: Unfreeze deeper layers
for layer in base_model.layers:
    if 'conv4' in layer.name or 'conv5' in layer.name:
        layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [23]:
#Fine-Tune Training
model.fit(train_generator, epochs=30, validation_data=val_generator,
          callbacks=[early_stop], class_weight=class_weights_dict)


Epoch 1/30
64/64 [==============================] - 29s 400ms/step - loss: 0.1653 - accuracy: 0.9590 - val_loss: 0.3874 - val_accuracy: 0.8668
Epoch 2/30
64/64 [==============================] - 24s 379ms/step - loss: 0.1101 - accuracy: 0.9738 - val_loss: 0.4679 - val_accuracy: 0.8549
Epoch 3/30
64/64 [==============================] - 25s 395ms/step - loss: 0.0943 - accuracy: 0.9778 - val_loss: 0.4306 - val_accuracy: 0.8529
Epoch 4/30
64/64 [==============================] - 25s 393ms/step - loss: 0.0749 - accuracy: 0.9842 - val_loss: 0.4283 - val_accuracy: 0.8748
Epoch 5/30
64/64 [==============================] - 26s 400ms/step - loss: 0.0571 - accuracy: 0.9896 - val_loss: 0.3769 - val_accuracy: 0.8787
Epoch 6/30
64/64 [==============================] - 25s 385ms/step - loss: 0.0521 - accuracy: 0.9876 - val_loss: 0.4237 - val_accuracy: 0.8648
Epoch 7/30
64/64 [==============================] - 25s 386ms/step - loss: 0.0430 - accuracy: 0.9906 - val_loss: 0.4965 - val_accuracy: 0.8608

In [24]:
#Evaluation
loss, acc = model.evaluate(val_generator)
print(f"Test Accuracy: {acc * 100:.2f}%")


16/16 [==============================] - 5s 286ms/step - loss: 0.3835 - accuracy: 0.8787
Test Accuracy: 87.87%


In [ ]:
# Predictions
# True labels from validation set
y_true = val_generator.classes  

# Predicted probabilities
y_pred_probs = model.predict(val_generator)

# Convert probabilities -> class indices
y_pred = np.argmax(y_pred_probs, axis=1)


In [ ]:
# Classification Report
class_labels = list(val_generator.class_indices.keys())
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))


Classification Report:

              precision    recall  f1-score   support

   cardboard       0.96      0.81      0.88        80
       glass       0.88      0.88      0.88       100
       metal       0.81      0.93      0.86        82
       paper       0.91      0.95      0.93       118
     plastic       0.86      0.86      0.86        96
       trash       0.77      0.63      0.69        27

    accuracy                           0.88       503
   macro avg       0.87      0.84      0.85       503
weighted avg       0.88      0.88      0.88       503



In [ ]:
# # # # Save Model
# model.save("resnet_model_fixed.h5")

c:\Users\Begad\anaconda3\envs\tf27\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
